This is my first kernel.  I am new to machine learning so any help would be much appreciated! I created a kernel with a random forest regressor because these regressors are a great place to start and have the following advantages:

 + Limited parameter tuning ... default parameters often have great performance
 + Fast
 + Versitile
 + Great for feature selection because it evaluates a lot of decision tree variations

Additionally, I used a pipeline to help with the data preparation steps:
 + selector - Converst DataFrame to a NumPy array.
 + Imputer - There is a lot of missing data in this set, so NaN's are replaced with median
 + std_scaler - Not needed for decision trees, but a good step for future algorithms
 + KBest - Selects the most important features
 + PCA - Transforms the features into correlated components
 + Random Forest Regressor - Used to make predictions

Finally, I ran a grid search to tune the pipeline paramaters for KBest, PCA, and the Random Forest Regressor

In [4]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [5]:
train_df = pd.read_csv("train_2016_v2.csv", parse_dates=["transactiondate"])

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 3 columns):
parcelid           90275 non-null int64
logerror           90275 non-null float64
transactiondate    90275 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 2.1 MB


In [7]:
test_df = pd.read_csv("sample_submission.csv")

In [8]:
test_df = test_df.rename(index=str, columns={"ParcelId": "parcelid"})

In [9]:
properties_2016_df = pd.read_csv("properties_2016.csv")

C:\Users\203014767\.conda\envs\DAND\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train_df = pd.merge(train_df, properties_2016_df, on='parcelid', how='left')

In [11]:
test_df = pd.merge(test_df, properties_2016_df, on='parcelid', how='left')

### Prepare the data for Machine Learning algorithms

In [21]:
train_df_pipeline = train_df

In [12]:
# Let's remove the top and bottom 5% of outliers
outlier_upper = np.percentile(train_df.logerror.values, 95)
outlier_lower = np.percentile(train_df.logerror.values, 5)
train_df['logerror'].ix[train_df['logerror']>outlier_upper] = outlier_upper
train_df['logerror'].ix[train_df['logerror']<outlier_lower] = outlier_lower

C:\Users\203014767\.conda\envs\DAND\lib\site-packages\pandas\core\indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
cat_df_train = pd.concat([train_df['propertyzoningdesc'], train_df['propertycountylandusecode'],
                          train_df['taxdelinquencyflag'],train_df['fireplaceflag'], 
                          train_df['hashottuborspa']], axis = 1)

In [14]:
numeric_df_train = train_df.drop(['propertyzoningdesc', 'propertycountylandusecode', 'transactiondate', 'parcelid',
                         'taxdelinquencyflag','hashottuborspa', 'fireplaceflag', 'logerror'], axis = 1)

In [15]:
# Create a Y value to predict for our training set
train_df_labels = train_df['logerror'].values

In [49]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [50]:
# this class is used to help with the pipeline.  
# More details on this class can found in chapter 2 of Hands-on Machine Learning with Scikit-Learn and Tensorflow
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [51]:
# Create a pipeline for quicker data preparation
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, recall_score
from sklearn.ensemble import RandomForestRegressor

num_attribs = list(numeric_df_train)

feature_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),        
        ('KBest', SelectKBest(k = 10)),
        ('pca', PCA(n_components = 5)),
        ('reg', RandomForestRegressor(random_state=42))
         ])

The CategoricalEncoder expects a 2D array containing one or more categorical input features. We need to reshape housing_cat to a 2D array:

In [ ]:
# Create parameters to random search to find an optimal pipeline
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
sss = StratifiedShuffleSplit(100, test_size=0.5, random_state=42)
param_distribs = {
    'KBest__k' : randint(low=1, high=53),
    'pca__n_components' : randint(low=1, high=30),
    }

In [ ]:
# Create the random search
from sklearn.model_selection import RandomizedSearchCV
#Try from sklearn.cross_validation import StratifiedKFold... keeps balancing constant

rnd_search = RandomizedSearchCV(feature_pipeline, param_distributions=param_distribs,
                                n_iter=3, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [ ]:
# Fit the random search to the training set
rnd_search_fit = rnd_search.fit(numeric_df_train,train_df_labels)

In [ ]:
# {'KBest__k': 43, 'pca__n_components': 8}
rnd_search_fit.best_params_

In [ ]:
features = rnd_search_fit.best_params_

In [ ]:
# save model for future use
from sklearn.externals import joblib

joblib.dump(features, "features.pkl")

In [52]:
train_df = train_df.drop(['logerror', 'transactiondate', 'parcelid'], axis = 1)

In [53]:
train_df = train_df.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [54]:
num_attribs = list(numeric_df_train)
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),        
        ('KBest', SelectKBest(k = 43)),
        ('pca', PCA(n_components = 8))
])

In [55]:
cat_attribs = list(cat_df_train)
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
    ])     

In [56]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [57]:
train_df_prepared = full_pipeline.fit_transform(train_df,train_df_labels)
train_df_prepared

array([[-0.90822706,  1.40833849,  1.28894048, ...,  1.        ,
         1.        ,  1.        ],
       [ 2.5796806 , -0.11577835, -1.95326955, ...,  1.        ,
         1.        ,  1.        ],
       [-0.08292109,  1.90345618,  0.69711428, ...,  1.        ,
         1.        ,  1.        ],
       ..., 
       [-1.07166518,  1.41754263,  0.68052192, ...,  1.        ,
         1.        ,  1.        ],
       [-1.82597089,  0.87639533,  0.75194357, ...,  1.        ,
         1.        ,  1.        ],
       [-2.58620329,  0.30440888, -0.05544064, ...,  1.        ,
         1.        ,  1.        ]])

In [58]:
# save model for future use
from sklearn.externals import joblib

joblib.dump(train_df_prepared, "train_df_prepared_zillow7.pkl")

[u'train_df_prepared_zillow7.pkl']

In [2]:
from sklearn.externals import joblib
train_df_prepared = joblib.load("train_df_prepared_zillow7.pkl")

In [ ]:
# Download training labels
joblib.dump(train_df_labels, "train_df_labels_zillow7.pkl")

In [16]:
# split the training set into a test and train set
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split = \
    train_test_split(train_df_prepared, train_df_labels, test_size=0.3, random_state=42)

In [25]:
# Create parameters to random search to find an optimal pipeline
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
sss = StratifiedShuffleSplit(100, test_size=0.5, random_state=42)
param_distribs = {
    'min_samples_split': randint(low=1, high=100),
    'n_estimators': randint(low=1, high=2000),
    'max_depth': randint(low=1, high=30)
    }

In [26]:
# Create the random search
from sklearn.model_selection import RandomizedSearchCV
#Try from sklearn.cross_validation import StratifiedKFold... keeps balancing constant
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)

rnd_search_rf = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=3, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [ ]:
# Fit the random search to the training set
rnd_search_rf = rnd_search_rf.fit(X_train, y_train)

In [ ]:
# return the best parameters of the random search
rnd_search_rf.best_params_

In [ ]:
best_parameters = rnd_search.best_params_

In [ ]:
# return the best model
rnd_search.best_estimator_

In [ ]:
final_model = rnd_search.best_estimator_

In [ ]:
# review each of the 5 searchs of the random search and their respective scores
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
param_distribs_svm = {
        {'kernel': ['linear'], 'C': randint(low=10, high=3000)},
        {'kernel': ['rbf'], 'C':randint(low=10, high=1000),
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    }

In [ ]:
# save model for future use
from sklearn.externals import joblib

joblib.dump(final_model, "final_model_zillow7.pkl")

In [ ]:
# save paramaters for future use
joblib.dump(best_parameters, "best_param_zillow7.pkl")

In [ ]:
# Run final predictions
RF_rand_final_predictions_test = final_model.predict(test_df)

In [86]:
submission_file = pd.read_csv('sample_submission.csv') 
for column in submission_file.columns[submission_file.columns != 'ParcelId']:
    submission_file[column] = RF_rand_final_predictions_test

In [87]:
submission_file.to_csv('RF_final_predictions.csv', index=False, float_format='%.4f')